In [5]:
import json
import uuid
from functools import reduce

import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display, HTML

from jinja2 import Template

# Define a quartely time schema
years_in_model = 3
quarters = range(1, years_in_model * 4 + 1)
quarter_labels = ['Q' + str(q) for q in quarters]
#### STEP 1:  Calculate simple non-discounted cost-benefit cash flows for a single quarter with a delay ramp, and max
#### STEP 2:  Use those cash flows to find a discounted cash flow for each project using Net Present Value (NPV)
#### STEP 3:  Assemble the net present value of estimated future cash flows across projects in the portfolio
#### STEP 4:  Show a composite overall return, and let the user change paramaters, risk levels, timing, etc. to optimize and plan

In [6]:
class CashFlow():
    def __init__(self, delay_qtrs, is_cost, max_amt, scale_up_qtrs, function, name='', flow_id=uuid.uuid4(), tot_qtrs=12):
        if scale_up_qtrs < 2: 
            raise 'the total number of quarters must be at least one'

        self.id = flow_id
        self.is_cost = is_cost
        self.function = function # Will interpret an instance according to this setting
        self.delay_qtrs = delay_qtrs
        self.name = name
        self.max_amt = max_amt
        self.scale_up_qtrs = scale_up_qtrs
        self.tot_qtrs = tot_qtrs
        
    def _sigmoid(self, x):
        """
        We define y at 95% max at end of delay and scale up period 
        y = .95L = L / (1 + e^-k(x_end - x_naught)) # https://en.wikipedia.org/wiki/Logistic_function
        For us, this means the scaling factor (k) is
        k = ln(1/.95 - 1)/((delay_qtrs + scale_up_qtrs/2) - (delay_qtrs + scale_up_qtrs))
        """
        # 1 / 2 scale_up_qtrs to place x_naught at vertical of sigmoid from after delay
        x_naught = self.delay_qtrs + self.scale_up_qtrs / 2
        x_end = self.delay_qtrs + self.scale_up_qtrs
        k = math.log(1/.95 - 1) / (x_naught - x_end)
        return self.max_amt / (1 + math.exp(-k * (x - x_naught)))
    
    def _linear(self, x):
        """y = mx + b. Units in amount (returned value) per quarter (x)"""
        m = self.max_amt / self.scale_up_qtrs
        b = -m * self.delay_qtrs
        return min(m * x + b, self.max_amt) # Never return more than max
    
    def _single(self, x):
        if x == self.delay_qtrs:
            return self.max_amt
        else:
            return 0
    
    def _step(self, x):
        return self.max_amt
    
    def _calculate_qtr(self, f):
        values = []
        for quarter_n in range(0, self.tot_qtrs):
            if quarter_n < self.delay_qtrs:
                values.append(0)
            else:
                multiplier = -1 if self.is_cost else 1
                # TODO: multiply by -1 here if it is a COST we are considering
                values.append(multiplier * f(quarter_n))
        return values

    def quick_view(self):
        fig = plt.figure()
        ax = fig.add_subplot(1, 1, 1)
        ax.plot(range(self.tot_qtrs), self.sigmoid_qtr, label='sigmoid')
        ax.plot(range(self.tot_qtrs), self.linear_qtr, label='linear')
        ax.plot(range(self.tot_qtrs), self.step_qtr, label='step')
        ax.scatter(range(self.tot_qtrs), self.single_qtr, label='single')
        ax.legend(loc='upper left')
        ax.grid(True)
    
    @property
    def qtr(self):
        """calculates quarter for instance based on set function type"""
        return self._calculate_qtr(getattr(self, f'_{self.function.lower()}'))
    
    @property
    def sigmoid_qtr(self):
        return self._calculate_qtr(self._sigmoid)
    
    @property
    def linear_qtr(self):
        return self._calculate_qtr(self._linear)
    
    @property
    def step_qtr(self):
        return self._calculate_qtr(self._step)
    
    @property
    def single_qtr(self):
        return self._calculate_qtr(self._single)
    
    def to_json(self):
        return {
            "flow_id": str(self.id),
            "function": self.function,
            "delay_qtrs": self.delay_qtrs,
            "is_cost": self.is_cost,
            "name": self.name,
            "max_amt": self.max_amt,
            "scale_up_qtrs": self.scale_up_qtrs,
            "tot_qtrs": self.tot_qtrs,
        }

In [7]:
# cf = CashFlow(name='test', delay_qtrs = 3, max_amt = 5, scale_up_qtrs = 7, function='linear', is_cost=False)

# def pp(name, lst):
#     print(f"\n{name}")
#     print('\t'.join(["{0:.2f}".format(i) for i in lst]))
    
# pp('SIGMOID', cf.sigmoid_qtr)
# pp('LINEAR', cf.linear_qtr)
# pp('SINGLE', cf.single_qtr)
# pp('STEP', cf.step_qtr)

# cf.quick_view()

# Porfolio Model Tool

This tool allows you to define projects with a number of cash flows. To add a new project, edit the file to add a new project:
```
batch_performance = Project('batch_performance')
```

after adding it there, you must also add it to the list of projects.
```
projects = [batch_performance, r2k]
```

Once that is done you are able to look at the cash flows of all projects combined. As well as for any given project.


In [14]:
class Styles():
    BORDER_RIGHT = "border-right: 1px solid #CDCDCD;"
    BORDER_BOTTOM = "border-bottom: 1px solid #CDCDCD;"
    
class Project():
    """Add cash flows to a project"""
    def __init__(self, name):
        self.cash_flows = []
        self.name = name  # define before attempting to load from disk
        self._load_from_disk()  # important to load before generating cash_flow_list
        self.cash_flow_list = widgets.VBox(children=[self.widgetize_cf(cf) for cf in self.cash_flows])

        self.is_cost = widgets.Checkbox(value=False, description='Check if cash flow is a cost')
        self.cf_name = widgets.Text(placeholder='Name or description', description='Name')
        self.func_type = widgets.ToggleButtons(options=['linear', 'sigmoid', 'single', 'step'], description='Form')
        self.ben_delay = widgets.IntSlider(min=0, max=8, step=1, value=2, description='Ben. Delay', continuous_update=False)
        self.ben_ramp = widgets.FloatSlider(min=2, max=12, step=0.5, value=4, description='Ben. Ramp', continuous_update=False)
        self.ben_max = widgets.FloatSlider(min=0, max=30, step=0.5, value=4, description='Ben. Max', continuous_update=False)
        
        create_btn = widgets.Button(description="Add Cash Flow")
        create_btn.on_click(self.on_create_click)

        self.graph_out = widgets.Output()
        self.update_graphs()
        
        menu = widgets.VBox(
            children=[self.cf_name, self.is_cost, self.func_type, self.ben_delay, self.ben_ramp,
                      self.ben_max, create_btn, self.graph_out],
            layout=widgets.Layout(overflow="hidden")
        )
        
        self.app = widgets.AppLayout(left_sidebar=menu, right_sidebar=self.cash_flow_list)
        display(self.app)

    def _on_delete_click_factory(self, model):
        """A partially applied function that accepts a model,
        and returns a new function a button can click with context
        to a specific model"""

        def on_delete_click(button):
            index = None
            for i, cash_flow in enumerate(self.cash_flows):
                if cash_flow.id == model.id:
                    index = i
                    break

            if index is not None:
                self.cash_flows.pop(index)
                self.update_cash_flows()
                self.update_graphs()

        return on_delete_click
    
    def _save_to_disk(self):
        with open(self.filename, 'w') as outfile:
            json.dump([cf.to_json() for cf in self.cash_flows], outfile)
    
    def _load_from_disk(self):
        try:
            with open(self.filename) as json_file:
                data = json.load(json_file)
                if not isinstance(data, list):
                    raise Exception(f'delete the file: {self.filename}, data is bad. It will regenerate.')
                for d in data:
                    cf = CashFlow(**d)
                    self.cash_flows.append(cf)
        except FileNotFoundError:
            print('could not find file, creating new')
    
    @property
    def filename(self):
        slug = self.name.lower().replace(' ', '-')
        return f'./data/{slug}-data.json'
        
    def update_cash_flows(self):
        self.cash_flow_list.children = [self.widgetize_cf(cf) for cf in self.cash_flows]
        self._save_to_disk()

    def on_create_click(self, event):
        f = CashFlow(
            delay_qtrs = self.ben_delay.value, 
            max_amt = self.ben_max.value,
            scale_up_qtrs = self.ben_ramp.value,
            function = self.func_type.value,
            name = self.cf_name.value,
            is_cost = self.is_cost.value
        )
        self.cash_flows.append(f)
        self.update_cash_flows()
        self.update_graphs()

    def widgetize_cf(self, model):
        html = f"""
        <h6 style="text-align: center; margin: 3px auto;">{model.name}</h1>
        <table style="margin: auto; text-align: center;">
            <tr>
                <th style="padding: 2px; {Styles.BORDER_BOTTOM} {Styles.BORDER_RIGHT}">Function</th>
                <th style="padding: 2px; {Styles.BORDER_BOTTOM} {Styles.BORDER_RIGHT}">Is Cost</th>
                <th style="padding: 2px; {Styles.BORDER_BOTTOM} {Styles.BORDER_RIGHT}">Delay Quarters</th>
                <th style="padding: 2px; {Styles.BORDER_BOTTOM} {Styles.BORDER_RIGHT}">Max Amount</th>
                <th style="padding: 2px; {Styles.BORDER_BOTTOM}">Scale Time</th>
            </tr>
            <tr>
                <td style="padding: 2px; {Styles.BORDER_RIGHT}">{model.function}</td>
                <td style="padding: 2px; {Styles.BORDER_RIGHT}">{model.is_cost}</td>
                <td style="padding: 2px; {Styles.BORDER_RIGHT}">{model.delay_qtrs}</td>
                <td style="padding: 2px; {Styles.BORDER_RIGHT}">{model.max_amt}</td>
                <td style="padding: 2px;">{model.scale_up_qtrs}</td>
            </tr>
        </table>
        """
        html = widgets.HTML(
            value=html,
            layout=widgets.Layout(width='100%')
        )
        delete_button = widgets.Button(
            description='delete',
            button_style='danger',
            layout=widgets.Layout(margin='5px'),
        )
        delete_button.on_click(self._on_delete_click_factory(model))
        return widgets.VBox(
            children=[html, delete_button],
            layout=widgets.Layout(
                border='1px solid',
                margin='4px',
                justify_content='center',
                align_items='center',
                display='flex'
            )
        )
    
    def update_graphs(self):
        self.graph_out.clear_output(wait=True)
        if len(self.cash_flows) == 0:
            return

        def trim_axs(axs, N):
            axs = axs.flat
            for ax in axs[N:]:
                ax.remove()
            return axs[:N]

        def get_min_max(flows):
            values = reduce(list.__add__, [flow.qtr for flow in flows])
            return min(0, min(values)), max(values)
        
        # capture stdout of matplotlib to output catching context graph_out as widget to include
        with self.graph_out:
            cols = 3
            rows = len(self.cash_flows) // cols + 1
            figsize = (10, 8)
            fig, axs = plt.subplots(rows, cols, figsize=figsize, constrained_layout=True)
            axs = trim_axs(axs, len(self.cash_flows))
            ymin, ymax = get_min_max(self.cash_flows)
            for ax, cf in zip(axs, self.cash_flows):
                ax.set_title(cf.name)
                ax.set_ylim([ymin,ymax])
                ax.grid(True)
                ax.xaxis.set_ticks(range(1, 13))
                ax.plot(range(12), cf.qtr)

            plt.show(fig)


# Batch Performance

In [15]:
batch_performance = Project('batch_performance')

AppLayout(children=(VBox(children=(Text(value='', description='Name', placeholder='Name or description'), Chec…

# R2K

In [72]:
r2k = Project('r2k')

AppLayout(children=(VBox(children=(Text(value='', description='Name', placeholder='Name or description'), Chec…

In [63]:
projects = [batch_performance, r2k]
graph_out = widgets.Output()

def combine_flows(flows):
    qtr_values = map(lambda cf: cf.qtr, flows)
    aggregated_values = [sum(values) for values in zip(*qtr_values)]
    return aggregated_values

def get_x_ticks(index, bar_width, step=1):
    return [x - (bar_width * len(projects)) + (bar_width * index) for x in range(12)]

def graph_projects_cash_flow(a):
    graph_out.clear_output(wait=True)
    
    with graph_out:
        projects_values = map(lambda p: combine_flows(p.cash_flows), projects)
        projects_net = [sum(values) for values in zip(*projects_values)]
        
        figsize = (25, 8)
        plt.figure(figsize=figsize)
        plt.xticks(range(0, 12))
        bar_room = .75 # 75% of space will have a bar plotted on it
        bar_width = bar_room * 1 / (len(projects) + 1)  # + 1 in denominator to account for 'net'
        legend_parents = []
        for i, project in enumerate(projects):
            xseries = get_x_ticks(i, bar_width)
            plot = plt.bar(xseries, combine_flows(project.cash_flows), width=bar_width)
            legend_parents.append(plot)

        legend_parents = [p[0] for p in legend_parents]
        net_xseries = get_x_ticks(len(projects), bar_width)
        net_plot = plt.bar(net_xseries, projects_net, color='black', width=bar_width)

        legend_parents.append(net_plot[0])
        plt.legend((*legend_parents,), (*map(lambda p: p.name, projects), 'net'))
        plt.show()

with graph_out:
    graph_projects_cash_flow(None)

button = widgets.Button(description='Refresh Chart')
button.on_click(graph_projects_cash_flow)

chart = widgets.VBox(children=[button, graph_out])
chart

## Step 1

In [8]:
### STEP 1 START ###
# For logistic growth we can define a sigmoid function
# Sigmoid function, f(-6) = 0; f(0) = 0.5;  f(6) = 1.0;
# Sigmoid / logistic growth would be a common cash flow profile
# Future options to add would be 'step function,' 'linear growth,' 'exponential growth,' etc.
def sigmoid(x):
    shift = 3 # 5% of max value is "zero"
    return 1 / (1 + math.exp(-(x - shift)))

# Define a quarterly implementation of logistic growth / sigmoid based on delay in years, max $ amount, and the time to ramp up
# All values in quarters except the 'max_amt' which is in dollars($M USD)
# max_amt is the 'carrying capacity' of the sigmoid function, usually known as 'k' in academic formulas
def sigmoid_qtr(delay_qtrs, max_amt, scale_up_qtrs, tot_qtrs=12):
    mylist = []
    if scale_up_qtrs < 2: 
        print('the total number of quarters must be at least one')
    # -3 corresponds to 5% of max value, and 3 corresponds to 95% of max value, going to use those values to define function.
    sigmoid_start = -3
    sigmoid_last = 3
    quarterly_x = (sigmoid_last - sigmoid_start) / scale_up_qtrs
    # print(f'quarterly_x = {quarterly_x}')
    for i in range(0, tot_qtrs):
        if i < delay_qtrs:
            mylist.append(0)
        else: 
            x_val = sigmoid_start + quarterly_x*(i - delay_qtrs)
            mylist.append(max_amt * sigmoid(x_val))
    return mylist

# Let's try a test case with some benefits with a max of $5M that start in 3 quarters, fully realized in 6 quarters
delay_qtrs = 3
max_amt = 5
scale_up_qtrs = 7
test = sigmoid_qtr(delay_qtrs, max_amt, scale_up_qtrs)

# Red line is the number of delay quarters, when the function kicks in
# Blue line is the end of scale up when you are getting close to max benefits
def plotter(ben_vals, delay_qtrs, scale_up_qtrs):
    labels = np.arange(1, len(test)+1)
    plt.bar(labels, ben_vals)
    plt.axvline(x=delay_qtrs, color = 'r')
    plt.title('Scale up of Cash Flows')
    plt.axvline(x=(delay_qtrs + scale_up_qtrs), color = 'b')
    plt.show;
# plotter(test, delay_qtrs, scale_up_qtrs)

# This should print both costs and benefits
def dual_plotter(costs, bennies):

    # set width of bar
    barWidth = 0.25

    # calculate the net benefit for the quarter
    net = [x1 - x2 for (x1, x2) in zip(bennies, costs)]
    
    # set height of bar
    bars1 = costs
    bars2 = bennies
    bars3 = net

    # Set position of bar on X axis
    r1 = np.arange(len(bars1))
    r2 = [x + barWidth for x in r1]
    r3 = [x + barWidth for x in r2]

    # Make the plot
    plt.bar(r1, bars1, color='r', width=barWidth, edgecolor='white', label='cost')
    plt.bar(r2, bars2, color='b', width=barWidth, edgecolor='white', label='bennies')
    plt.bar(r3, bars3, color='#2d7f5e', width=barWidth, edgecolor='white', label='net')

    # Add xticks on the middle of the group bars
    plt.xlabel('group', fontweight='bold')
    plt.xticks([r + barWidth for r in range(len(bars1))], quarter_labels)

    # Create legend & Show graphic
    plt.legend()
    plt.show();


In [9]:
def f(x, y, z, a, b, c):
    benefits = sigmoid_qtr(x, y, z)
    costs = sigmoid_qtr(a, b, c)
    
    # To plot a single cash flow:
    # plotter(benefits, x, y, costs)
    
    # To plot both costs and benefits together:
    dual_plotter(benefits, costs)


# Need to get this working for negative numbers.  Cash flow profiles for costs / investments will be negative numbers.
# But maybe we can just calculate as positive, and then take the diff for net cash flow and put into a dataframe
# Just need to review math of the above functions to make sure it works for negative numbers.
# Turning off continuous update here :) 
benefits_delay = widgets.IntSlider(min=0, max=8, step=1, value=2, description='Ben. Delay', continuous_update=False)
benefits_ramp = widgets.FloatSlider(min=2, max=12, step=0.5, value=4, description='Ben. Ramp', continuous_update=False)
benefits_max = widgets.FloatSlider(min=0, max=30, step=0.5, value=4, description='Ben. Max', continuous_update=False)

# We should probably include a component eventually where costs are a function of benefits, but for now, let's model them independently.
cost_delay = widgets.FloatSlider(min=0, max=12, step=0.5, value=4, description='Cost Delay', continuous_update=False)
cost_ramp = widgets.FloatSlider(min=0, max=12, step=0.5, value=4, description='Cost Ramp', continuous_update=False)
cost_max = widgets.FloatSlider(min=0, max=30, step=0.5, value=4, description='Cost Max', continuous_update=False)

interact(f, x=benefits_delay, y=benefits_ramp, z=benefits_max, a=cost_delay, b=cost_ramp, c=cost_max);


##### There is something weird in how the sigmoid_qtr function is returning values.  
##### Example, if you set the 'cost max' or 'benefit max' really high, it seems to change the ramp-up time, 
##### and the max value in the slider doesn't match the max value in the plot.

interactive(children=(IntSlider(value=2, continuous_update=False, description='Ben. Delay', max=8), FloatSlide…

# Step 2

In [10]:
## Discounted Cash Flows can use this formula, once we find the : 
example_net_cashflow = [-100, 19, 49, 58, 200]

# Using a shortcut here dividing by four to calculate quarterly interest from annual discount rate of 25%
# There is a better way to do the conversion, but example below:
npv_net =  np.npv(0.35/4,[-100, 19, 49, 58, 200, 100]) 
print("Net Present Value(npv) : ", npv_net) 

# we could also calculate an IRR using the IRR module from python, but NPV is fine for now.

#### STEP 2 END
#### STEP 3 START

Net Present Value(npv) :  212.7356691470436


In [11]:
## Start building out a dataframe for project financial parameters and 12 months of cash flows
## Cash flows can be populated with the growth curves above

row_titles = ['visibility', 'growth', 'gamechangers']
column_titles = ['internal_fte', 'external_spend', 'max_benefit', 'profile', 'discount_rate']
d = pd.DataFrame(0, index=row_titles, columns=column_titles)

In [13]:
years_in_model = 3
quarters = (list(range(years_in_model * 4)))
quarters = [q + 1 for q in quarters]

In [14]:
a = np.zeros((len(row_titles),len(quarters)))
cost_benefit = pd.DataFrame(a, index = row_titles, columns=quarters)

In [15]:
# so once we estimate the benefits and the costs, we can populate the quarterly cash flows into a dataframe
# The three major categories of projects are visibility (R2K, etc.), growth (batch perf., etc.), gamechangers (proteinview, etc.)
comb = pd.concat([d, cost_benefit], axis=1, sort=False)
comb

# Once we phase in cash flows, we can discount them back with an Net Present Value (NPV) using the discount rate.

,internal_fte,external_spend,max_benefit,profile,discount_rate,1,2,3,4,5,6,7,8,9,10,11,12
visibility,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
growth,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
gamechangers,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
### HMM... how to iterate over the rows to replace the cash flows in the data frame for a given investment area.
# adder = 1
# for key, value in cost_benefit.iteritems(): 
#     value = value + adder
#     adder += 1
#     print(value) 
#     print() 